## 1차 전처리
1. 11년 데이터 제거
2. 결산월 12월 이외의 데이터 제거
3. 상장 이전 회계연도 데이터 제거
4. 금융업 제거
5. 한글 데이터 처리
6. 수치형 데이터 데이터 타입 숫자형으로 통일

In [1]:
import pandas as pd

df = pd.read_csv('../data/dataset/코스피_전처리전.csv')
df.head()

,Symbol,Name,결산월,회계년,주기,당기순이익률(%),매출총이익률(%),영업이익률(%),자본금영업이익률(%),자본금세전계속사업이익률(%),...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,벤포드,분식기업
0,A005930,삼성전자,12,2011,Annual,8.34,32.03,9.48,"1,743.07","1,915.50",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-0.05,0,0.0
1,A005930,삼성전자,12,2012,Annual,11.86,37.02,14.44,"3,236.64","3,333.10",...,0.89,0.87,0.98,1.22,0.01,1.00,0.94,-0.05,0,0.0
2,A005930,삼성전자,12,2013,Annual,13.33,39.79,16.08,"4,098.54","4,274.50",...,0.92,0.93,0.93,1.14,0.01,1.05,0.91,-0.05,1,0.0
3,A005930,삼성전자,12,2014,Annual,11.35,37.79,12.14,"2,788.27","3,105.80",...,1.10,1.05,1.15,0.90,0.01,1.08,0.90,-0.05,0,0.0
4,A005930,삼성전자,12,2015,Annual,9.50,38.46,13.16,"2,942.96","2,892.54",...,1.05,0.98,0.85,0.97,0.01,0.99,0.96,-0.06,0,0.0


In [2]:
print(df.shape, df.drop_duplicates('Name').shape)

(13356, 48) (1100, 48)


---
### 1. 11년 데이터 제거

In [3]:
df = df[df['회계년'] != 2011]

In [4]:
print(df.shape, df.drop_duplicates('Name').shape)

(12243, 48) (1100, 48)


---
### 2. 결산월 12월 이외의 데이터 제거

In [5]:
df = df[df['결산월'] == 12]

In [6]:
print(df.shape, df.drop_duplicates('Name').shape)

(12150, 48) (1099, 48)


---
### 3. 상장 이전 회계연도 데이터 제거 (불필요한 데이터 제거)

In [7]:
# KRX 내 상장일 데이터 (코스피)

kospi_info = pd.read_csv('../data/코스피_기본정보.csv', encoding='CP949')
kospi_info = kospi_info[['단축코드', '한글 종목약명', '상장일']]

# '상장일' 칼럼을 날짜 형식으로 변환
kospi_info['상장일'] = pd.to_datetime(kospi_info['상장일'], format='%Y/%m/%d')

# 연도만 남기기
kospi_info['상장일'] = kospi_info['상장일'].dt.year

# '상장일' 칼럼을 문자열로 변환
kospi_info['상장일'] = kospi_info['상장일'].astype(str)

# '단축코드' 칼럼의 모든 데이터에 'A'를 추가하기
kospi_info['단축코드'] = 'A' + kospi_info['단축코드'].astype(str)

# '단축코드' 칼럼명을 'Symbol'로 변경하기
kospi_info.rename(columns={'단축코드':'Symbol', 
                           '한글 종목약명':'Name', 
                           '상장일':'상장연도'}, inplace=True)

In [8]:
df = pd.merge(kospi_info, df, on='Symbol')

In [9]:
df.head()

,Symbol,Name_x,상장연도,Name_y,결산월,회계년,주기,당기순이익률(%),매출총이익률(%),영업이익률(%),...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,벤포드,분식기업
0,A095570,AJ네트웍스,2015,AJ네트웍스,12,2012,Annual,2.90,33.81,13.57,...,0.78,0.76,4.13,1.38,0.00,1.57,0.96,-0.03,1,0.0
1,A095570,AJ네트웍스,2015,AJ네트웍스,12,2013,Annual,2.39,100.00,9.32,...,0.63,0.34,0.31,4.13,0.01,4.48,1.03,0.12,0,0.0
2,A095570,AJ네트웍스,2015,AJ네트웍스,12,2014,Annual,2.77,100.00,7.61,...,0.81,1.00,1.07,1.34,0.18,1.02,0.97,0.11,0,0.0
3,A095570,AJ네트웍스,2015,AJ네트웍스,12,2015,Annual,2.75,100.00,7.03,...,1.03,1.00,0.81,1.04,0.18,1.01,0.96,0.10,0,0.0
4,A095570,AJ네트웍스,2015,AJ네트웍스,12,2016,Annual,1.36,100.00,4.93,...,1.13,1.00,1.28,1.19,0.17,1.02,1.03,0.12,0,0.0


In [10]:
# '상장연도' 칼럼의 데이터 타입을 정수형으로 변환
df['상장연도'] = df['상장연도'].astype(int)

# '회계년' 칼럼이 '상장연도' 칼럼보다 작은 행을 필터링하여 삭제
df = df[df['회계년'] >= df['상장연도']]

# 위 코드는 '회계년' 칼럼이 '상장연도' 칼럼보다 크거나 같은 행만 유지하고, 나머지 행은 모두 삭제합니다. 
# 이렇게 하면 '회계년'이 '상장연도'보다 작은 행은 필터링되어 제거됩니다.

In [11]:
df.head()

,Symbol,Name_x,상장연도,Name_y,결산월,회계년,주기,당기순이익률(%),매출총이익률(%),영업이익률(%),...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,벤포드,분식기업
3,A095570,AJ네트웍스,2015,AJ네트웍스,12,2015,Annual,2.75,100.0,7.03,...,1.03,1.0,0.81,1.04,0.18,1.01,0.96,0.10,0,0.0
4,A095570,AJ네트웍스,2015,AJ네트웍스,12,2016,Annual,1.36,100.0,4.93,...,1.13,1.0,1.28,1.19,0.17,1.02,1.03,0.12,0,0.0
5,A095570,AJ네트웍스,2015,AJ네트웍스,12,2017,Annual,2.45,100.0,2.66,...,1.96,1.0,8.47,0.67,0.14,1.02,1.02,0.07,0,0.0
6,A095570,AJ네트웍스,2015,AJ네트웍스,12,2018,Annual,3.56,100.0,-2.03,...,0.53,1.0,0.44,1.24,0.05,1.05,1.00,0.02,0,0.0
7,A095570,AJ네트웍스,2015,AJ네트웍스,12,2019,Annual,4.21,100.0,1.56,...,1.36,1.0,1.80,0.96,0.07,0.96,1.00,0.03,0,0.0


In [12]:
df[df['Name_x'] != df['Name_y']]

,Symbol,Name_x,상장연도,Name_y,결산월,회계년,주기,당기순이익률(%),매출총이익률(%),영업이익률(%),...,DSRI,GMI,AQI,SGI,DEPI,SGAI,LVGI,TATA,벤포드,분식기업
2787,A286940,롯데이노베이트,2018,롯데정보통신,12,2018,Annual,2.88,8.74,4.80,...,0.18,0.90,0.73,5.44,0.00,0.91,0.77,-0.10,1,0.0
2788,A286940,롯데이노베이트,2018,롯데정보통신,12,2019,Annual,6.27,8.84,4.86,...,0.85,0.99,1.07,1.04,0.01,1.01,0.92,-0.11,0,0.0
2789,A286940,롯데이노베이트,2018,롯데정보통신,12,2020,Annual,3.52,8.11,4.56,...,0.90,1.09,0.94,1.00,0.01,0.89,0.83,0.08,0,0.0
2790,A286940,롯데이노베이트,2018,롯데정보통신,12,2021,Annual,3.73,8.13,4.30,...,1.15,1.00,1.26,1.09,0.01,1.08,1.16,-0.06,1,0.0
2791,A286940,롯데이노베이트,2018,롯데정보통신,12,2022,Annual,2.71,8.94,3.27,...,1.26,0.91,1.58,1.13,0.01,1.48,1.13,0.02,0,0.0
4892,A010620,HD현대미포,1983,현대미포조선,12,2012,Annual,1.98,8.12,2.13,...,1.57,1.74,0.95,0.95,0.02,1.02,0.99,0.11,0,0.0
4893,A010620,HD현대미포,1983,현대미포조선,12,2013,Annual,-6.70,0.08,-6.90,...,1.41,96.95,0.82,0.90,0.02,1.17,1.22,0.04,0,0.0
4894,A010620,HD현대미포,1983,현대미포조선,12,2014,Annual,-17.12,-13.42,-21.87,...,0.33,-0.01,0.63,1.00,0.02,1.21,1.18,-0.13,0,0.0
4895,A010620,HD현대미포,1983,현대미포조선,12,2015,Annual,0.55,8.73,1.43,...,2.14,-1.54,1.06,1.17,0.02,0.86,1.00,0.14,0,0.0
4896,A010620,HD현대미포,1983,현대미포조선,12,2016,Annual,1.15,10.11,5.54,...,0.21,0.86,0.91,0.74,0.01,0.63,0.93,-0.04,0,0.0


In [13]:
df.drop(['Name_y'], axis=1, inplace=True)
df.rename(columns={'Name_x': 'Name'}, inplace=True)

In [14]:
print(df.shape, df.drop_duplicates('Name').shape)

(7751, 49) (778, 49)


---
### 4. 금융업 제거 (불필요한 데이터 제거)
- WICS 기준 금융업 제외
- 은행 / 증권 / 다각화된금융 / 보험 / 부동산
- 키워드 제거

In [15]:
# 은행
bank = ['KB금융', 
        '신한지주', 
        '하나금융지주', 
        '우리금융지주', 
        '기업은행', 
        '카카오뱅크', 
        'BNK금융지주', 
        'JB금융지주', 
        'DGB금융지주', 
        '상상인', 
        '제주은행']

# result = result[~result['Name'].isin(bank)]

# 증권
securities = ['메리츠금융지주', 
              '한국금융지주', 
              '미래에셋증권', 
              '삼성증권', 
              'NH투자증권', 
              '키움증권', 
              '다우기술', 
              '대신증권', 
              '한화투자증권', 
              '유진투자증권', 
              '신영증권', 
              '유안타증권', 
              'SK증권', 
              '다우데이타', 
              '다올투자증권', 
              '교보증권', 
              '현대차증권', 
              'DB금융투자', 
              '케이프', 
              '한양증권', 
              '부국증권', 
              '이베스트투자증권', 
              '유화증권', 
              '상상인증권', 
              '코리아에셋투자증권']

# result = result[~result['Name'].isin(securities)]

# 다각화된금융
diversified = ['삼성카드', 
               '우리기술투자', 
               '스틱인베스트먼트', 
               '글로벌텍스프리', 
               '미래에셋벤처투자', 
               '아주IB투자', 
               '에이티넘인베스트', 
               '리드코프', 
               'DSC인베스트먼트', 
               'SV인베스트먼트', 
               'SBI인베스트먼트', 
               '대성창투', 
               '한국캐피탈', 
               "메이슨캐피탈", 
               'HB인베스트먼트', 
               '캡스톤파트너스', 
               '스톤브릿지벤처스', 
               '컴퍼니케이', 
               '나우IB', 
               '큐캐피탈', 
               'TS인베스트먼트', 
               '푸른저축은행', 
               'LB인베스트먼트', 
               '리더스 기술투자', 
               '린드먼아시아']

# result = result[~result['Name'].isin(diversified)]

insurance = ['삼성화재', 
             '삼성생명', 
             'DB손해보험', 
             '현대해상', 
             '한화생명', 
             '코리안리', 
             '한화손해보험', 
             '미래에셋생명', 
             '롯데손해보험', 
             '동양생명', 
             '인카금융서비스', 
             '에이플러스에셋', 
             '흥국화재']

# result = result[~result['Name'].isin(insurance)]

df = df[~df['Name'].isin(set(bank)|set(securities)|set(diversified)|set(insurance))]

In [16]:
df = df[~df['Name'].str.contains('은행|증권|투자|인베스트먼트|스팩|보험|화재|신탁|금융|카드|캐피탈|우리종금|대성창투')]

In [17]:
print(df.shape, df.drop_duplicates('Name').shape)

(7313, 49) (732, 49)


In [18]:
df.reset_index(drop=True, inplace = True)

In [19]:
df.isnull().sum()

Symbol                 0
Name                   0
상장연도                   0
결산월                    0
회계년                    0
주기                     0
당기순이익률(%)              1
매출총이익률(%)              1
영업이익률(%)               1
자본금영업이익률(%)           45
자본금세전계속사업이익률(%)       45
*영업수익/영업비용(%)          0
ROE(세전계속사업이익)(%)      45
자본금지배주주순이익률(%)        45
매출액증가율(전년동기)(%)       47
총자본증가율(전년동기)(%)       45
유형자산증가율(전년동기)(%)      45
영업이익증가율(전년동기)(%)      45
당기순이익증가율(전년동기)(%)     45
유형자산구성비율(%)            0
무형자산구성비율(%)            0
*판관비율(%)               1
타인자본비율(%)              0
부채비율(%)                0
현금및현금성자산구성비율(%)        0
총자산회전율(회)             45
총부채회전율(회)             45
총자본회전율(회)             45
자본금회전율(회)             45
현금흐름/총부채(%)           45
현금흐름/총차입부채(%)        277
현금흐름/영업수익(%)           1
현금흐름/총자본(%)           45
유동비율(%)                0
당좌비율(%)                0
매출채권회전율               45
재고자산회전율               45
고정자산회전율               45
영업현금흐름/투자현금흐름(%)       0
DSRI                  68


---
### 5. 한글 데이터 처리

In [20]:
df.columns[6:-2]

Index(['당기순이익률(%)', '매출총이익률(%)', '영업이익률(%)', '자본금영업이익률(%)', '자본금세전계속사업이익률(%)',
       '*영업수익/영업비용(%)', 'ROE(세전계속사업이익)(%)', '자본금지배주주순이익률(%)',
       '매출액증가율(전년동기)(%)', '총자본증가율(전년동기)(%)', '유형자산증가율(전년동기)(%)',
       '영업이익증가율(전년동기)(%)', '당기순이익증가율(전년동기)(%)', '유형자산구성비율(%)', '무형자산구성비율(%)',
       '*판관비율(%)', '타인자본비율(%)', '부채비율(%)', '현금및현금성자산구성비율(%)', '총자산회전율(회)',
       '총부채회전율(회)', '총자본회전율(회)', '자본금회전율(회)', '현금흐름/총부채(%)', '현금흐름/총차입부채(%)',
       '현금흐름/영업수익(%)', '현금흐름/총자본(%)', '유동비율(%)', '당좌비율(%)', '매출채권회전율',
       '재고자산회전율', '고정자산회전율', '영업현금흐름/투자현금흐름(%)', 'DSRI', 'GMI', 'AQI', 'SGI',
       'DEPI', 'SGAI', 'LVGI', 'TATA'],
      dtype='object')

In [21]:
import re

columns_to_check = df.columns[6:-2]

# 각 열에 대해 고유한 값 추출하여 한글을 포함하는 값 확인
for column in columns_to_check:
    # 고유한 값 추출
    unique_values = df[column].astype(str).unique()
    # 한글을 포함하는 값 추출
    korean_values = [value for value in unique_values if re.search('[ㄱ-ㅎㅏ-ㅣ가-힣]', value)]
    print(f"'{column}'에 존재하는 문자 데이터: {korean_values}")

'당기순이익률(%)'에 존재하는 문자 데이터: []
'매출총이익률(%)'에 존재하는 문자 데이터: []
'영업이익률(%)'에 존재하는 문자 데이터: []
'자본금영업이익률(%)'에 존재하는 문자 데이터: []
'자본금세전계속사업이익률(%)'에 존재하는 문자 데이터: []
'*영업수익/영업비용(%)'에 존재하는 문자 데이터: []
'ROE(세전계속사업이익)(%)'에 존재하는 문자 데이터: ['완전잠식']
'자본금지배주주순이익률(%)'에 존재하는 문자 데이터: []
'매출액증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'총자본증가율(전년동기)(%)'에 존재하는 문자 데이터: ['당기잠식', '전기잠식', '잠식지속']
'유형자산증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'영업이익증가율(전년동기)(%)'에 존재하는 문자 데이터: ['적전', '흑전', '적지']
'당기순이익증가율(전년동기)(%)'에 존재하는 문자 데이터: ['적전', '흑전', '적지']
'유형자산구성비율(%)'에 존재하는 문자 데이터: []
'무형자산구성비율(%)'에 존재하는 문자 데이터: []
'*판관비율(%)'에 존재하는 문자 데이터: []
'타인자본비율(%)'에 존재하는 문자 데이터: []
'부채비율(%)'에 존재하는 문자 데이터: ['완전잠식']
'현금및현금성자산구성비율(%)'에 존재하는 문자 데이터: []
'총자산회전율(회)'에 존재하는 문자 데이터: []
'총부채회전율(회)'에 존재하는 문자 데이터: []
'총자본회전율(회)'에 존재하는 문자 데이터: ['완전잠식']
'자본금회전율(회)'에 존재하는 문자 데이터: []
'현금흐름/총부채(%)'에 존재하는 문자 데이터: []
'현금흐름/총차입부채(%)'에 존재하는 문자 데이터: []
'현금흐름/영업수익(%)'에 존재하는 문자 데이터: []
'현금흐름/총자본(%)'에 존재하는 문자 데이터: ['완전잠식']
'유동비율(%)'에 존재하는 문자 데이터: []
'당좌비율(%)'에 존재하는 문자 데이터: []
'매출채권회전율'에

- 'ROE(세전계속사업이익)(%)'에 존재하는 문자 데이터: ['완전잠식']
- '총자본증가율(전년동기)(%)'에 존재하는 문자 데이터: ['당기잠식', '전기잠식', '잠식지속']
- '영업이익증가율(전년동기)(%)'에 존재하는 문자 데이터: ['적전', '흑전', '적지']
- '당기순이익증가율(전년동기)(%)'에 존재하는 문자 데이터: ['적전', '흑전', '적지']
- '부채비율(%)'에 존재하는 문자 데이터: ['완전잠식']
- '총자본회전율(회)'에 존재하는 문자 데이터: ['완전잠식']
- '현금흐름/총자본(%)'에 존재하는 문자 데이터: ['완전잠식']

In [22]:
value = pd.read_csv('../data/dataset/코스피_연결재무제표계정값.csv')
value = pd.merge(df, value, on = ['Symbol', 'Name', '결산월', '회계년', '주기'], how = 'left')

value['영업이익(천원)_전기'] = value.groupby('Name')['영업이익(천원)'].shift(1)
value['당기순이익(천원)_전기'] = value.groupby('Name')['당기순이익(천원)'].shift(1)
value['자본총계_전기'] = value.groupby('Name')['총자본(천원)'].shift(1)

In [23]:
# ROE(세전계속사업이익)(%): (세전계속사업이익 / 자본총계 * 100
df['ROE(세전계속사업이익)(%)'] = df['ROE(세전계속사업이익)(%)'].mask(df['ROE(세전계속사업이익)(%)'] == '완전잠식',
                                                     round((value['세전계속사업이익(천원)']/value['총자본(천원)'])*100, 2))

# 총자본증가율: 자본총계[당기]/자본총계[전기]-1
df['총자본증가율(전년동기)(%)'] = df['총자본증가율(전년동기)(%)'].mask(df['총자본증가율(전년동기)(%)'].isin
                                                   (['당기잠식', '전기잠식', '잠식지속']),
                                                   round(((value['총자본(천원)']-value['자본총계_전기'])/value['자본총계_전기'])*100, 2))


# 영업이익증가율: (영업이익[당기]-영업이익[전기])/영업이익[전기]
df['영업이익증가율(전년동기)(%)'] = df['영업이익증가율(전년동기)(%)'].mask(df['영업이익증가율(전년동기)(%)'].isin
                                                     (['적전', '흑전', '적지']),
                                       round(((value['영업이익(천원)']-value['영업이익(천원)_전기'])/value['영업이익(천원)_전기'])*100, 2))

# 순이익증가율: (당기순이익[당기]-당기순이익[전기])/당기순이익[전기]
df['당기순이익증가율(전년동기)(%)'] = df['당기순이익증가율(전년동기)(%)'].mask(df['당기순이익증가율(전년동기)(%)'].isin
                                                       (['적전', '흑전', '적지']),
                                       round(((value['당기순이익(천원)']-value['당기순이익(천원)_전기'])/value['당기순이익(천원)_전기'])*100, 2))


# 부채비율: 부채총계/자본총계
df['부채비율(%)'] = df['부채비율(%)'].mask(df['부채비율(%)'] == '완전잠식', round((value['총부채(천원)']/value['총자본(천원)'])*100, 2))

# 총자본회전율, 현금흐름/총자본 0으로 대체
df['총자본회전율(회)'] = df['총자본회전율(회)'].mask(df['총자본회전율(회)'] == '완전잠식', 0)
df['현금흐름/총자본(%)'] = df['현금흐름/총자본(%)'].mask(df['현금흐름/총자본(%)'] == '완전잠식', 0)

In [24]:
df['영업이익증가율(전년동기)(%)']

0         -3.53
1         -16.7
2        -63.73
3       -194.64
4       -173.69
         ...   
7308     -13.82
7309      10.37
7310     -53.11
7311     -67.66
7312   -1683.16
Name: 영업이익증가율(전년동기)(%), Length: 7313, dtype: object

In [25]:
import re

columns_to_check = df.columns[6:-2]

# 각 열에 대해 고유한 값 추출하여 한글을 포함하는 값 확인
for column in columns_to_check:
    # 고유한 값 추출
    unique_values = df[column].astype(str).unique()
    # 한글을 포함하는 값 추출
    korean_values = [value for value in unique_values if re.search('[ㄱ-ㅎㅏ-ㅣ가-힣]', value)]
    print(f"'{column}'에 존재하는 문자 데이터: {korean_values}")

'당기순이익률(%)'에 존재하는 문자 데이터: []
'매출총이익률(%)'에 존재하는 문자 데이터: []
'영업이익률(%)'에 존재하는 문자 데이터: []
'자본금영업이익률(%)'에 존재하는 문자 데이터: []
'자본금세전계속사업이익률(%)'에 존재하는 문자 데이터: []
'*영업수익/영업비용(%)'에 존재하는 문자 데이터: []
'ROE(세전계속사업이익)(%)'에 존재하는 문자 데이터: []
'자본금지배주주순이익률(%)'에 존재하는 문자 데이터: []
'매출액증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'총자본증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'유형자산증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'영업이익증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'당기순이익증가율(전년동기)(%)'에 존재하는 문자 데이터: []
'유형자산구성비율(%)'에 존재하는 문자 데이터: []
'무형자산구성비율(%)'에 존재하는 문자 데이터: []
'*판관비율(%)'에 존재하는 문자 데이터: []
'타인자본비율(%)'에 존재하는 문자 데이터: []
'부채비율(%)'에 존재하는 문자 데이터: []
'현금및현금성자산구성비율(%)'에 존재하는 문자 데이터: []
'총자산회전율(회)'에 존재하는 문자 데이터: []
'총부채회전율(회)'에 존재하는 문자 데이터: []
'총자본회전율(회)'에 존재하는 문자 데이터: []
'자본금회전율(회)'에 존재하는 문자 데이터: []
'현금흐름/총부채(%)'에 존재하는 문자 데이터: []
'현금흐름/총차입부채(%)'에 존재하는 문자 데이터: []
'현금흐름/영업수익(%)'에 존재하는 문자 데이터: []
'현금흐름/총자본(%)'에 존재하는 문자 데이터: []
'유동비율(%)'에 존재하는 문자 데이터: []
'당좌비율(%)'에 존재하는 문자 데이터: []
'매출채권회전율'에 존재하는 문자 데이터: []
'재고자산회전율'에 존재하는 문자 데이터: []
'고정자산회전율'에 존재하는 문자 데이터: []
'영업현금흐름

---
### 6. feature 컬럼 데이터타입 숫자형으로 변환

In [26]:
for col in df.columns[6:-2]:
    df[col] = df[col].replace(',', '', regex=True)  # 콤마 제거
    df[col] = pd.to_numeric(df[col], errors='coerce')

In [27]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7313 entries, 0 to 7312
Data columns (total 49 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Symbol             7313 non-null   object 
 1   Name               7313 non-null   object 
 2   상장연도               7313 non-null   int32  
 3   결산월                7313 non-null   int64  
 4   회계년                7313 non-null   int64  
 5   주기                 7313 non-null   object 
 6   당기순이익률(%)          7312 non-null   float64
 7   매출총이익률(%)          7312 non-null   float64
 8   영업이익률(%)           7312 non-null   float64
 9   자본금영업이익률(%)        7268 non-null   float64
 10  자본금세전계속사업이익률(%)    7268 non-null   float64
 11  *영업수익/영업비용(%)      7313 non-null   float64
 12  ROE(세전계속사업이익)(%)   7268 non-null   float64
 13  자본금지배주주순이익률(%)     7268 non-null   float64
 14  매출액증가율(전년동기)(%)    7266 non-null   float64
 15  총자본증가율(전년동기)(%)    7262 non-null   float64
 16  유형자산증가율(전년동기)(%)   7268 

In [28]:
print(df.shape, df.drop_duplicates('Name').shape)

(7313, 49) (732, 49)


In [29]:
# df.to_csv('코스피_1차전처리후.csv',index = False, encoding = 'UTF-8-sig')

In [30]:
df.isnull().sum()

Symbol                 0
Name                   0
상장연도                   0
결산월                    0
회계년                    0
주기                     0
당기순이익률(%)              1
매출총이익률(%)              1
영업이익률(%)               1
자본금영업이익률(%)           45
자본금세전계속사업이익률(%)       45
*영업수익/영업비용(%)          0
ROE(세전계속사업이익)(%)      45
자본금지배주주순이익률(%)        45
매출액증가율(전년동기)(%)       47
총자본증가율(전년동기)(%)       51
유형자산증가율(전년동기)(%)      45
영업이익증가율(전년동기)(%)     189
당기순이익증가율(전년동기)(%)    252
유형자산구성비율(%)            0
무형자산구성비율(%)            0
*판관비율(%)               1
타인자본비율(%)              0
부채비율(%)                0
현금및현금성자산구성비율(%)        0
총자산회전율(회)             45
총부채회전율(회)             45
총자본회전율(회)             45
자본금회전율(회)             45
현금흐름/총부채(%)           45
현금흐름/총차입부채(%)        277
현금흐름/영업수익(%)           1
현금흐름/총자본(%)           45
유동비율(%)                0
당좌비율(%)                0
매출채권회전율               45
재고자산회전율               45
고정자산회전율               45
영업현금흐름/투자현금흐름(%)       0
DSRI                  68
